In [1]:
import requests
import time

In [2]:
def request_url(url):
    response = requests.get(url)
    while(response.status_code != 200):
        response = requests.get(url)
        time.sleep(1)
    response = response.json()
    return response

In [3]:
def get_discover_tracks_id(CLIENT_ID):
    result = set()
    
    url = f'https://api-v2.soundcloud.com/mixed-selections?client_id={CLIENT_ID}&limit=100&linked_partitioning=1'
    while(1):
        response = request_url(url)
        collections = response['collection']    
        for collection in collections:
            items = collection['items']['collection']
            for item in items:
                try:
                    tracks = item['tracks']
                    for track in tracks:
                        result.add(track['id'])
                except:
                    break
        try:
            url = response['next_href']+ '&client_id='+CLIENT_ID
        except:
            break
    return result

In [4]:
def get_discover_playlists_id(CLIENT_ID):
    result = []
    temp=set()
    url = f'https://api-v2.soundcloud.com/mixed-selections?client_id={CLIENT_ID}&limit=100&linked_partitioning=1'
    while(1):
        response = request_url(url)
        collections = response['collection']    
        for collection in collections:
            items = collection['items']['collection']
            for item in items:
                try:
                    uri = item['uri']
                    temp.add(uri)
                except:
                    break
        try:
            url = response['next_href']+ '&client_id='+CLIENT_ID
        except:
            break
    new_temps=list(temp)
    for new_temp in new_temps:
        result.append(new_temp[37:])
    return result

In [5]:
def extract_playlist_generals(playlist_id, client_id):
    url = f'https://api-v2.soundcloud.com/playlists/{playlist_id}?client_id={client_id}&limit=9&linked_partitioning=1'
    while True:
        time.sleep(1)
        response = requests.get(url)
        if not response.ok:
            print(f"Failed {response.url}")
            continue
        break

    print(f"Hit {response.url}")
    generals_data = response.json()
    generals_data['user'] = generals_data['user']['id']
    print(generals_data['user'])
    tracks_list = []
    for track in generals_data['tracks']:
        tracks_list.append(track['id'])
    generals_data['tracks'] = tracks_list
    return generals_data

In [6]:
def playlist_info(playlist_id, client_id):
    reposters_url = f'https://api-v2.soundcloud.com/playlists/{playlist_id}/reposters?client_id={client_id}&limit=9&linked_partitioning=1'
    likers_url = f'https://api-v2.soundcloud.com/playlists/{playlist_id}/likers?client_id={client_id}&limit=9&linked_partitioning=1'
    generals_data = extract_playlist_generals(playlist_id, client_id)
    generals_data['reposters'] = get_id_from_collection(reposters_url, client_id, 100)
    generals_data['likers'] = get_id_from_collection(likers_url, client_id, 100)


In [7]:
def get_id_from_collection(url, client_id, result_limit):
    results = set()
    full = False
    while True:
        response = requests.get(url)
        if not response.ok:
            print(f"Failed {response.url}")
            continue
        print(f"Hit {response.url}")
        try:
            json_data = response.json()
            collections = json_data['collection']
            if not collections:
                break
            for collection in collections:
                if len(results) < result_limit or result_limit==-1 :
                    results.add(collection['id'])
                else:
                    full = True
                    break
            if full or json_data['next_href'] is None:
                break
            url = json_data['next_href'] + f'&client_id={client_id}'
        except KeyError:
            break
    return results

In [8]:
def get_id_from_collection_with_option(url, client_id,option, result_limit):
    results = set()
    full = False
    while True:
        response = requests.get(url)
        if not response.ok:
            print(f"Failed {response.url}")
            continue
        print(f"Hit {response.url}")
        try:
            json_data = response.json()
            collections = json_data['collection']
            if not collections:
                break
            for collection in collections:
                if len(results) < result_limit or result_limit==-1:
                    results.add(collection[option]['id'])
                else:
                    full = True
                    break
            if full or json_data['next_href'] is None:
                break
            url = json_data['next_href'] + f'&client_id={client_id}'
        except KeyError:
            break
    return results

In [9]:
def extract_track_generals(track_id, client_id):
    url = f'https://api-v2.soundcloud.com/tracks/{track_id}?client_id={client_id}&linked_partitioning=1'
    while True:
        time.sleep(1)
        response = requests.get(url)
        if not response.ok:
            print(f"Failed {response.url}")
            continue
        break

    print(f"Hit {response.url}")
    generals_data = response.json()
    generals_data['user'] = generals_data['user']['id']
    print(generals_data['user'])

    return generals_data


In [10]:
def track_info(track_id, client_id):
    reposters_url = f'https://api-v2.soundcloud.com/tracks/{track_id}/reposters?client_id={client_id}&limit=9&linked_partitioning=1'
    likers_url = f'https://api-v2.soundcloud.com/tracks/{track_id}/likers?client_id={client_id}&limit=9&linked_partitioning=1'
    related_tracks_url = f'https://api-v2.soundcloud.com/tracks/{track_id}/related?client_id={client_id}&limit=10&linked_partitioning=1'
    album_url = f'https://api-v2.soundcloud.com/tracks/{track_id}/albums?&client_id={client_id}&limit=1&linked_partitioning=1'
    playlists_url =f'https://api-v2.soundcloud.com/tracks/{track_id}/playlists_without_albums?client_id={client_id}&limit=10&linked_partitioning=1'
    comments_url = f'https://api-v2.soundcloud.com/tracks/{track_id}/comments?threaded=0&filter_replies=0&client_id={client_id}&limit=20&linked_partitioning=1'
    
    generals_data = extract_track_generals(track_id, client_id)
    generals_data['reposters'] = get_id_from_collection(reposters_url, client_id, 100)
    generals_data['likers'] = get_id_from_collection(likers_url, client_id, 100)
    generals_data['comments'] =get_id_from_collection_with_option(comments_url, client_id,'user', 100)
    generals_data['related_tracks'] = get_id_from_collection(related_tracks_url, client_id, 100)
    generals_data['album'] = get_id_from_collection(album_url, client_id, 100)
    generals_data['playlists'] = get_id_from_collection(playlists_url, client_id, 100)
    return generals_data

In [11]:
def get_discover(client_id):
    
    tracks = get_discover_tracks_id(client_id)
    playlist = get_discover_playlists_id(client_id)
    
    track_result = []
    playlist_result = []
    
    for track_id in tracks:
        track_result.append(track_info(track_id, client_id))
    for playlist_id in playlists:
        playlist_result.append(playlist_info(playlist_id, client_id))
    return track_result,playlist_result

In [12]:
def extract_user_generals(user_id, client_id):
    url =f'https://api-v2.soundcloud.com/users/{user_id}?client_id={client_id}&limit=100&linked_partitioning=1'
    while True:
        time.sleep(1)
        response = requests.get(url)
        if not response.ok:
            print(f"Failed {response.url}")
            continue
        break

    print(f"Hit {response.url}")
    generals_data['user'] = response.json()

    print(generals_data['user'])

    return generals_data

In [13]:
def user_info(user_id,client_id):
    
    general_url =f'https://api-v2.soundcloud.com/users/{user_id}?client_id={client_id}&limit=100&linked_partitioning=1'
    feautured_profile_url =f'https://api-v2.soundcloud.com/users/{user_id}/featured-profiles?client_id={client_id}&limit=10&linked_partitioning=1'
    web_profile_url=f'https://api-v2.soundcloud.com/users/soundcloud:users:{user_id}/web-profiles?client_id={client_id}'
    spotlight_url=f'https://api-v2.soundcloud.com/users/{user_id}/spotlight?client_id={client_id}&limit=20&linked_partitioning=1'
    user_tracks_url=f'https://api-v2.soundcloud.com/users/{user_id}/tracks?client_id={client_id}&limit=20&linked_partitioning=1'
    user_top_tracks_url=f'https://api-v2.soundcloud.com/users/{user_id}/toptracks?client_id={client_id}&linked_partitioning=1'
    user_albums_url=f'https://api-v2.soundcloud.com/users/{user_id}/albums?client_id={client_id}&limit=10&linked_partitioning=1'
    user_playlist_without_albums_url=f'https://api-v2.soundcloud.com/users/{user_id}/playlists_without_albums?client_id={client_id}&limit=10&linked_partitioning=1'
    related_artist_url=f'https://api-v2.soundcloud.com/users/{user_id}/relatedartists?client_id={client_id}&limit=12&linked_partitioning=1'
    reposts_url=f'https://api-v2.soundcloud.com/stream/users/{user_id}/reposts?client_id={client_id}&limit=10&linked_partitioning=1'
    likes_url=f'https://api-v2.soundcloud.com/users/{user_id}/likes?client_id={client_id}&limit=24&linked_partitioning=1'
    followings_url=f'https://api-v2.soundcloud.com/users/{user_id}/followings?client_id={client_id}&limit=3&linked_partitioning=1'
    followers_url=f'https://api-v2.soundcloud.com/users/{user_id}/followers?client_id={client_id}&limit=12&linked_partitioning=1'
    
    
    generals_data = extract_user_generals(user_id, client_id)
    generals_data['user_playlist_without_albums'] = get_id_from_collection(user_playlist_without_albums_url, client_id, 100)
    generals_data['followings'] = get_id_from_collection(followings_url, client_id, 100)
    generals_data['followers'] = get_id_from_collection(followers_url, client_id, 100)
    generals_data['reposts'] = get_id_from_collection_with_option(reposts_url, client_id,'track', 100)
    generals_data['likes'] = get_id_from_collection_with_option(likes_url, client_id,'track', 100)
    generals_data['feautured_profile'] = get_id_from_collection(feautured_profile_url, client_id, 100)
    generals_data['spotlight_tracks'] = get_id_from_collection(spotlight_url, client_id, 100)
    generals_data['user_tracks'] = get_id_from_collection(user_tracks_url, client_id, 100)
    generals_data['user_top_tracks'] = get_id_from_collection(user_top_tracks_url, client_id, 100)
    generals_data['user_albums'] = get_id_from_collection(user_albums_url, client_id, 100)
    generals_data['related_artist'] = get_id_from_collection(related_artist_url, client_id, 100)
    generals_data['web_profile']=request_url(web_profile_url)
    return generals_data